### Step 1 — Import Libraries

In [15]:
import pandas as pd
import numpy as np
import json
import os

# For resume text extraction
import fitz  

# For images (OCR dataset)
from PIL import Image

### Step 2 — Define dataset paths

In [16]:
DATA_PATH = r"data"

IBM_HR_PATH = r"C:\Guvi\Talent Intelligence & Workforce Optimization\data\ibm_hr"
RESUME_PATH = r"C:\Guvi\Talent Intelligence & Workforce Optimization\data\resumes"
JOBS_PATH = r"C:\Guvi\Talent Intelligence & Workforce Optimization\data\job_descriptions"
REVIEWS_PATH = r"C:\Guvi\Talent Intelligence & Workforce Optimization\data\employee_reviews"
DOC_OCR_PATH = r"C:\Guvi\Talent Intelligence & Workforce Optimization\data\document_ocr"


### Step 3 — Load IBM HR Dataset

In [17]:
hr_file = [f for f in os.listdir(IBM_HR_PATH) if f.endswith(".csv")][0]
hr_df = pd.read_csv(f"{IBM_HR_PATH}/{hr_file}")

print("HR Dataset Loaded:", hr_df.shape)
hr_df.head()


HR Dataset Loaded: (1470, 35)


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


###  Step 4 — Load Job Descriptions Dataset

In [18]:
jobs_files = [f for f in os.listdir(JOBS_PATH) if f.endswith(".csv") or f.endswith(".json")]

job_dfs = []
for file in jobs_files:
    path = f"{JOBS_PATH}/{file}"
    if file.endswith(".csv"):
        df = pd.read_csv(path)
    elif file.endswith(".json"):
        df = pd.read_json(path)
    job_dfs.append(df)

job_df = pd.concat(job_dfs, ignore_index=True)
print("Job Descriptions Loaded:", job_df.shape)
job_df.head()


Job Descriptions Loaded: (22000, 14)


,country,country_code,date_added,has_expired,job_board,job_description,job_title,job_type,location,organization,page_url,salary,sector,uniq_id
0,United States of America,US,NaN,No,jobs.monster.com,TeamSoft is seeing an IT Support Specialist to...,IT Support Technician Job in Madison,Full Time Employee,"Madison, WI 53702",NaN,http://jobview.monster.com/it-support-technici...,NaN,IT/Software Development,11d599f229a80023d2f40e7c52cd941e
1,United States of America,US,NaN,No,jobs.monster.com,The Wisconsin State Journal is seeking a flexi...,Business Reporter/Editor Job in Madison,Full Time,"Madison, WI 53708",Printing and Publishing,http://jobview.monster.com/business-reporter-e...,NaN,NaN,e4cbb126dabf22159aff90223243ff2a
2,United States of America,US,NaN,No,jobs.monster.com,Report this job About the Job DePuy Synthes Co...,Johnson & Johnson Family of Companies Job Appl...,"Full Time, Employee",DePuy Synthes Companies is a member of Johnson...,Personal and Household Services,http://jobview.monster.com/senior-training-lea...,NaN,NaN,839106b353877fa3d896ffb9c1fe01c0
3,United States of America,US,NaN,No,jobs.monster.com,Why Join Altec? If you’re considering a career...,Engineer - Quality Job in Dixon,Full Time,"Dixon, CA",Altec Industries,http://jobview.monster.com/engineer-quality-jo...,NaN,Experienced (Non-Manager),58435fcab804439efdcaa7ecca0fd783
4,United States of America,US,NaN,No,jobs.monster.com,Position ID# 76162 # Positions 1 State CT C...,Shift Supervisor - Part-Time Job in Camphill,Full Time Employee,"Camphill, PA",Retail,http://jobview.monster.com/shift-supervisor-pa...,NaN,Project/Program Management,64d0272dc8496abfd9523a8df63c184c


### Step 5 — Load Employee Reviews Dataset

In [19]:
review_file = [f for f in os.listdir(REVIEWS_PATH) if f.endswith(".csv")][0]
reviews_df = pd.read_csv(f"{REVIEWS_PATH}/{review_file}")

print("Reviews Loaded:", reviews_df.shape)
reviews_df.head()


Reviews Loaded: (355, 6)


,positionName,salary,company,rating,reviewsCount,jobTypeConsolidated
0,Pricing Analyst,"£70,000 - £85,000 a year",Urban Empire Recruitment,NaN,NaN,"Permanent, Full-time"
1,Lead AI / ML / Data Science Engineer,"£150,000 a year",Founding Teams,NaN,NaN,Part-time
2,Data Scientist,NaN,Depop,4.4,34.0,Permanent
3,Research Fellow in data analysis and machine l...,NaN,UCL,4.1,256.0,"Permanent, Full-time"
4,Senior Data Scientist | Media & Entertainment.,"£70,000 a year",Nicholson Glover Consulting,NaN,NaN,Permanent


### Step 6 — Extract Resume Text (PDF → TEXT)

In [20]:
# Step 6 (improved) — Recursively extract resume text from PDFs and combine with any CSV metadata
import os
import fitz       # PyMuPDF
import pdfplumber
import pandas as pd
from tqdm import tqdm

RESUME_PATH = r"C:\Guvi\Talent Intelligence & Workforce Optimization\data\resumes"   # adjust if your notebook path differs
OUTPUT_DIR = r"processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

pdf_files = []
# Walk folder recursively and collect PDFs
for root, dirs, files in os.walk(RESUME_PATH):
    for fname in files:
        if fname.lower().endswith(".pdf"):
            pdf_files.append(os.path.join(root, fname))

print(f"Found {len(pdf_files)} PDF files under: {RESUME_PATH}")

resume_records = []
corrupted_files = []

# Try extracting with PyMuPDF first; fallback to pdfplumber if necessary
for pdf_path in tqdm(pdf_files, desc="Extracting PDFs"):
    try:
        text = ""
        # Attempt PyMuPDF extraction
        try:
            doc = fitz.open(pdf_path)
            for page in doc:
                text += page.get_text()
            doc.close()
            if not text.strip():
                raise ValueError("PyMuPDF returned empty text")
        except Exception:
            # fallback to pdfplumber
            try:
                with pdfplumber.open(pdf_path) as pdf:
                    for page in pdf.pages:
                        text += page.extract_text() or ""
            except Exception as e:
                raise e

        resume_records.append({
            "filename": os.path.basename(pdf_path),
            "filepath": pdf_path,
            "text": text
        })
    except Exception as e:
        corrupted_files.append({"file": pdf_path, "error": str(e)})

print(f"Successfully extracted: {len(resume_records)}")
print(f"Failed / corrupted: {len(corrupted_files)}")
if corrupted_files:
    print("Example error:", corrupted_files[0])

resume_df = pd.DataFrame(resume_records)

# If there is a Resume.csv (or similar) with metadata, load and merge (best-effort)
csv_candidates = [p for p in os.listdir(RESUME_PATH) if p.lower().endswith(".csv")]
# Also check nested top-level folder named 'Resume' (some datasets use that)
for root, dirs, files in os.walk(RESUME_PATH):
    for f in files:
        if f.lower().endswith(".csv"):
            csv_candidates.append(os.path.join(root, f))

csv_candidates = list(dict.fromkeys(csv_candidates))  # unique
print("CSV metadata files found (if any):", csv_candidates)

if csv_candidates:
    # attempt to load the first CSV that looks like metadata
    csv_path = csv_candidates[0] if os.path.isabs(csv_candidates[0]) else os.path.join(RESUME_PATH, csv_candidates[0])
    try:
        meta_df = pd.read_csv(csv_path)
        print("Loaded resume metadata CSV:", csv_path, meta_df.shape)
        # Try to merge on filename (if filenames present in CSV)
        if 'filename' in meta_df.columns:
            merged = pd.merge(meta_df, resume_df, on='filename', how='left')
            final_df = merged
        else:
            # if CSV has a path column or name variations, keep them separate for now
            final_df = resume_df.copy()
            final_df = final_df.merge(meta_df, left_on='filename', right_on=meta_df.columns[0], how='left')
    except Exception as e:
        print("Failed to load CSV metadata:", e)
        final_df = resume_df.copy()
else:
    final_df = resume_df.copy()

# Clean text lightly (single-line)
import re
def clean_text(s):
    if isinstance(s, str):
        s = re.sub(r'\s+', ' ', s)
        return s.strip()
    return ""

final_df['clean_text'] = final_df['text'].apply(clean_text)

# Save result
out_path = os.path.join(OUTPUT_DIR, "resumes_clean.csv")
final_df.to_csv(out_path, index=False)
print("Saved cleaned resumes to:", out_path)
print("Final dataframe shape:", final_df.shape)


Found 2484 PDF files under: C:\Guvi\Talent Intelligence & Workforce Optimization\data\resumes


Extracting PDFs:   0%|          | 0/2484 [00:00<?, ?it/s]

Extracting PDFs: 100%|██████████| 2484/2484 [00:51<00:00, 48.29it/s]


Successfully extracted: 2484
Failed / corrupted: 0
CSV metadata files found (if any): ['C:\\Guvi\\Talent Intelligence & Workforce Optimization\\data\\resumes\\Resume\\Resume.csv']
Loaded resume metadata CSV: C:\Guvi\Talent Intelligence & Workforce Optimization\data\resumes\Resume\Resume.csv (2484, 4)
Failed to load CSV metadata: You are trying to merge on object and int64 columns for key 'filename'. If you wish to proceed you should use pd.concat
Saved cleaned resumes to: processed\resumes_clean.csv
Final dataframe shape: (2484, 4)


### Step 7 — Clean Text Columns (Jobs, Reviews, Resumes)

In [21]:
import re

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'\s+', ' ', text)   # remove multiple spaces/newlines
        text = text.replace('\n', ' ')
        return text.strip()
    return ""

# ==== Clean job description text ====
job_text_col = None

possible_cols = ['description', 'job_description', 'jobdesc', 'body', 'text']

for col in possible_cols:
    if col in job_df.columns:
        job_text_col = col
        break

print("Detected job description column:", job_text_col)

if job_text_col:
    job_df['clean_description'] = job_df[job_text_col].astype(str).apply(clean_text)
else:
    print("⚠️ No job description column found to clean.")


# ==== Clean employee review text ====

# Find the column that likely contains the review text
review_text_col = None

for col in reviews_df.columns:
    if reviews_df[col].dtype == 'object':   # choose first text column
        review_text_col = col
        break

print("Detected employee review text column:", review_text_col)

if review_text_col:
    reviews_df['clean_review'] = reviews_df[review_text_col].astype(str).apply(clean_text)
else:
    print("⚠️ No review text column found to clean.")


# ==== Clean resume extracted text (if needed) ====

if 'clean_text' not in resume_df.columns:
    resume_df['clean_text'] = resume_df['text'].astype(str).apply(clean_text)

print("Resume cleaning complete.")



Detected job description column: job_description
Detected employee review text column: positionName
Resume cleaning complete.


### Step 8 — Save Cleaned Versions

In [22]:
import os

os.makedirs("processed", exist_ok=True)

hr_df.to_csv("processed/hr_clean.csv", index=False)
job_df.to_csv("processed/job_descriptions_clean.csv", index=False)
reviews_df.to_csv("processed/employee_reviews_clean.csv", index=False)
resume_df.to_csv("processed/resumes_clean.csv", index=False)

print("All cleaned datasets saved!")


All cleaned datasets saved!
